<font size = 12>MCA and Measurements </font>

Author: Leonardo Assis Morais <br>

Use this notebook to: 
1. Control MCA to prepare TES system for measurements.
2. Perform measurements with TES system. 


***
<font size = 6, color = red > WARNING: </font> <br>

__You need to be connected at UQ VPN to properly use this notebook.__ <br>
__Login in the Cisco Any Connect Secure Mobility Client using <br>
your staff account and password.__ <br>
__IT must give you access to the TES system through UQ instruments <br>
network to use this notebook properly.__

***

Requires:

 - tes package from Geoff Gillett and Leo Assis.
 - bokeh (tested with version 2.3.2)

In [8]:
import numpy as np
import yaml
import serial.tools
import bokeh.io

import tes

bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
# for development/debugging of the tes package
%reload_ext autoreload

In [ ]:
from tes.data import fpga_stats

fpga_stats(1) # just to check if the FPGA is working properly

In [ ]:
from tes.registers import Registers

r = Registers('tcp://smp-loophole.instrument.net.uq.edu.au:10001')

# Running the MCA

In this section, we will configure the FPGA in order to <br>
perform the measurements. Run the subsections in order.

***

<font size = 6, color = red> WARNING: </font>

Remember of turning off the heatswitch *BEFORE* starting this <br>
procedure. The heatswitch power supply is an enormous source <br>
of noise. This procedure is described in the <br>
Transition Edge Sensors: A Practical Guide, by Leonardo Assis.

***

## Configuring the ADC channels

Analogue to Digital Converters (ADC's) are the devices responsible  <br>
to digitise the analogue output from the detectors. There are 8 ADCs <br>
and each can be connected to one of two processing channels. You need <br>
to check which ADC is connected when you are performing experiments.

In [ ]:
from tes.mca_control import configure_channels

adc_channel_A = 0
proc_channel_A = 0
invert_A = False

configure_channels(r, adc_channel_A, proc_channel_A, invert_A)

## Determining the detection baseline

In [ ]:
from tes.mca_control import baseline_offset

time = 2      # in seconds.
bin_width = 0 # bin width = 2^(bin_n)
baseline_offset = -1020 # adjust the offset

(fig,) = baseline_offset(r, time, proc_channel_A, bin_width,
                         baseline_offset)

ph = bokeh.plotting.show(fig, notebook_handle = True)

## Determining pulse threshold

In [21]:
from tes.mca_control import pulse_threshold

time = 1
p_thres = 0     
bin_width = 0    

(fig,) = pulse_threshold(r, time, proc_channel_A, bin_width, p_thres)

ph = bokeh.plotting.show(fig, notebook_handle = True)

## Determining the slope threshold

In [22]:
from tes.mca_control import slope_threshold

time = 1
s_thres = 60
bin_width = 0

(fig,) = slope_threshold(r, time, proc_channel_A, bin_width, s_thres)

ph = bokeh.plotting.show(fig, notebook_handle = True)

## Sanity Check: plotting an area histogram

In [24]:
from tes.mca_control import area_histogram

time = 10
bin_width = 10
xrange = 6e6
yrange = 5000
p_thres = 0
s_thres = 0

(fig,) = tes_mca.area_histogram(r, time, proc_channel_A, bin_width,
                                p_thres, s_thres, xrange, yrange)

ph = bokeh.plotting.show(fig, notebook_handle = True)

# Taking Measurements

***
<font color = red size = 6> WARNING: </font>

Remember of **determining both pulse and slope thresholds before <br>
taking measurements**. Taking measurements without having these variables <br>
defined **will** cause the FPGA to crash.

***

<font size=6, color = red>WARNING:</font>

Always remember to disable the MCA before performing measurements.
***

In [ ]:
r.mca.trigger='disabled'
r.mca.update

## Performing Measurements: Trace

trace_drive is a function to record trace measurements with the FPGA.
***
<font color = red size = 6> WARNING: </font>

Keep your trace measurements at a maximum of 1 minute. 
***

For more information on how to control the registers to perform <br>
measurements, see Geoff Gillett's thesis, A Hardware Processor <br>
for Transition-Edge sensors.

In [31]:
from tes.data_acquisition import trace_drive

time = 10 # in seconds

# choose location where the measurement registers will be saved
# these registers contains the FPGA configurations when your 
# measurements were performed
datapath = '/Users/leo/TES_package_and_data/Registers/'
filename = '2020-03-11-TES2'

# Registers details

p_thres = 1200 # pulse threshold
s_thres = 600  # slope threshold
baseline_sub = True # baseline subtraction

trace_drive(time, proc_channel_A, p_thres, s_thres,
               baseline_sub, datapath, filename)

40000.0
bl on
trace-pulse_BL-pulse_threshold-peak-test1
Frames captured:86 - dropped:0, invalid:0
Ticks:11, MCA histograms:0, Traces:25, Events:25


## Performing Measurements: Pulse Drive

Pulse drive is the function responsible to take multiple scalar <br>
measurements of the TES pulse in real time. 

In [40]:
from tes.data_acquisition import pulse_drive

time = 5*60 # in seconds

# location of saved registers

datapath = '/Users/leo/TES_package_and_data/Registers/'
filename = '2020-03-11-TES2'

# Registers details

p_thres = 800
s_thres = 600
baseline_sub = True

pulse_drive(time, proc_channel_A, p_thres, s_thres,
               baseline_sub, datapath, filename)

1200000.0
bl on
-pulse_BL-pulse_threshold-peak-test3
Frames captured:2399203 - dropped:0, invalid:0
Ticks:1200001, MCA histograms:5, Traces:0, Events:2863260


<font size = 12>END OF NOTEBOOK </font>